In [22]:
import os
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from abc import ABC, abstractmethod

In [23]:
docx_loader = DirectoryLoader("../../../preprocessing/DATA", glob="**/*.docx")

In [24]:
docs = docx_loader.load()

In [25]:
import re


def extract_info_from_filename(filename):
    """
    Input: filename ("MSFTTranscriptFY23Q4")
    Output: Extract stock symbol, year and quarter from filename
    """
    pattern = r"([A-Z]+)TranscriptFY(\d{2})Q(\d)"
    match = re.search(pattern, filename)

    if match:
        symbol = match.group(1)
        fiscal_year = match.group(2)
        fiscal_quarter = match.group(3)
        return symbol, fiscal_year, fiscal_quarter
    else:
        return None

In [26]:
db_name = "earning_calls"

In [27]:
def chunk_one_doc(doc):
    doc_chunks = []
    source = doc.metadata["source"]
    symbol, fiscal_year, fiscal_quarter = extract_info_from_filename(source)
    text_splitter = RecursiveCharacterTextSplitter(
        separators=[
            "\n## ",
            "\n### ",
            "\n#### ",
            "\n##### ",
            "\n###### ",
            "```\n\n",
            "\n\n***\n\n",
            "\n\n---\n\n",
            "\n\n___\n\n",
            "\n\n",
            "\n",
            " ",
            "",
        ]
    )

    chunks = text_splitter.split_text(doc.page_content)
    for i, chunk in enumerate(chunks):
        doc = Document(
            page_content=chunk,
            metadata={
                "source": source,
                "symbol": symbol,
                "fiscal_year": fiscal_year,
                "fiscal_quarter": fiscal_quarter,
                "chunk": i,
            },
        )
        doc_chunks.append(doc)
    return doc_chunks

In [28]:
from dotenv import dotenv_values
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
import openai
import pandas as pd
import numpy as np
import time
import requests

# specify the name of the .env file name
env_name = "../../../../.env"  # change to your own .env file name
config = dotenv_values(env_name)

if config["KEYS_FROM"] == "KEYVAULT":
    print("keyvault was selected.")
    keyVaultName = config["KEY_VAULT_NAME"]
    KVUri = f"https://{keyVaultName}.vault.azure.net"

    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=KVUri, credential=credential)

    openai.api_type = client.get_secret("OPENAI-API-TYPE").value
    openai.api_key = client.get_secret("OPENAI-API-KEY").value
    openai.api_base = client.get_secret("OPENAI-API-BASE").value
    openai.api_version = client.get_secret("OPENAI-API-VERSION").value
    deployment_embedding = client.get_secret("OPENAI-DEPLOYMENT-EMBEDDING").value
else:
    openai.api_type = config["OPENAI_API_TYPE"]
    openai.api_key = config["OPENAI_API_KEY"]
    openai.api_base = config["OPENAI_API_BASE"]
    openai.api_version = config["OPENAI_API_VERSION"]
    deployment_embedding = config["OPENAI_DEPLOYMENT_EMBEDDING"]


def createEmbeddings(text, endpoint, api_key, api_version, embedding_model_deployment):
    request_url = f"{endpoint}/openai/deployments/{embedding_model_deployment}/embeddings?api-version={api_version}"
    headers = {"Content-Type": "application/json", "api-key": api_key}
    request_payload = {"input": text}
    embedding_response = requests.post(
        request_url, json=request_payload, headers=headers, timeout=None
    )
    if embedding_response.status_code == 200:
        data_values = embedding_response.json()["data"]
        embeddings_vectors = [data_value["embedding"] for data_value in data_values]
        return embeddings_vectors
    else:
        raise Exception(f"failed to get embedding: {embedding_response.json()}")

keyvault was selected.


In [29]:
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
from dotenv import dotenv_values

config = dotenv_values(env_name)

if config["KEYS_FROM"] == "KEYVAULT":
    print("keyvault was selected.")
    keyVaultName = config["KEY_VAULT_NAME"]
    KVUri = f"https://{keyVaultName}.vault.azure.net"

    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=KVUri, credential=credential)
    MONGO_CONN = (client.get_secret("COSMOS-DB-MONGO-URI").value,)
else:
    print(".env was selected.")
    MONGO_CONN = config["COSMOS_DB_MONGO_URI"]

keyvault was selected.


In [30]:
import pymongo
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from abc import ABC, abstractmethod


class DatabaseService(ABC):
    @abstractmethod
    def store_data(self, data):
        pass

    @abstractmethod
    def retrieve_data(self, query, num_results):
        pass


SEARCH_INDEX_NAME = "vectorSearchIndex"


class MongoDBService(DatabaseService):
    def __init__(self, db_name, collection_name, search_index_name=SEARCH_INDEX_NAME):
        self.db_name = db_name
        self.collection_name = collection_name
        self.search_index_name = search_index_name
        self.client = MongoClient(MONGO_CONN)
        self.db = self.client[self.db_name]
        self.collection = self.db[self.collection_name]
        self._create_database()
        self._create_search_index()

    def _create_schema(self, docs):
        data = []
        for i, doc in enumerate(docs):
            # Create embeddings using the provided function
            embeddings = createEmbeddings(
                doc.page_content,
                openai.api_base,
                openai.api_key,
                openai.api_version,
                deployment_embedding,
            )[0]
            data.append(
                {
                    "id": i,
                    "content": doc.page_content,
                    "contentVector": embeddings,
                    "symbol": doc.metadata["symbol"],
                    "fiscal_year": doc.metadata["fiscal_year"],
                    "fiscal_quarter": doc.metadata["fiscal_quarter"],
                    "source": doc.metadata["source"],
                    "chunkid": doc.metadata["chunk"],
                }
            )
        return data

    def _create_database(self):
        # Check if the collection already exists, and create it if needed
        if self.collection_name not in self.db.list_collection_names():
            self.db.create_collection(self.collection_name)
            print("Created collection '{}'.".format(self.collection_name))
        else:
            print("Using collection: '{}'.".format(self.collection_name))

    def _create_search_index(self):
        self.db.command(
            {
                "createIndexes": self.collection_name,
                "indexes": [
                    {
                        "name": self.search_index_name,
                        "key": {"contentVector": "cosmosSearch"},
                        "cosmosSearchOptions": {
                            "kind": "vector-ivf",
                            "numLists": 1,
                            "similarity": "COS",  # TODO: Add other similarity options.
                            "dimensions": 1536,
                        },
                    }
                ],
            }
        )

    def _drop_data(self):
        self.collection.drop_index(self.search_index_name)
        self.client.drop_database(self.db_name)

    def store_data(self, data):
        try:
            # Insert data into the collection
            self.collection.insert_many(data)
            print("Data inserted successfully.")
        except Exception as e:
            print("Failed to insert data: ", e)

    def insert_one(self, entry):
        self.collection.insert_one(entry)

    def retrieve_data(self, query, num_results=1):
        try:
            # Perform a vector search query
            pipeline = [
                {
                    "$search": {
                        "cosmosSearch": {
                            "vector": query,
                            "path": "contentVector",
                            "k": num_results,
                        },
                        "returnStoredSource": True,
                    }
                }
            ]
            results = list(self.collection.aggregate(pipeline))
            return results
        except Exception as e:
            print("Search query failed: ", e)

In [31]:
client = MongoClient(MONGO_CONN)

In [32]:
client

MongoClient(host=['c.appliedaimongodb.mongocluster.cosmos.azure.com:10260'], document_class=dict, tz_aware=False, connect=True, tls=True, authmechanism='SCRAM-SHA-256', retrywrites=False, maxidletimems=120000)

In [33]:
for document in docs:
    chunks = chunk_one_doc(document)
    collection_name = (
        chunks[0].metadata["symbol"]
        + "FY"
        + chunks[0].metadata["fiscal_year"]
        + "Q"
        + chunks[0].metadata["fiscal_quarter"]
    )
    mongodb = MongoDBService(db_name=db_name, collection_name=collection_name)
    data = mongodb._create_schema(chunks)
    mongodb.store_data(data)

Using collection: 'MSFTFY23Q1'.
Data inserted successfully.
Using collection: 'MSFTFY23Q2'.
Data inserted successfully.
Using collection: 'MSFTFY23Q3'.
Data inserted successfully.
Using collection: 'MSFTFY23Q4'.
Data inserted successfully.


In [34]:
chunks[0].metadata["symbol"]

'MSFT'

In [35]:
from pymongo import MongoClient

mongo_client = MongoClient(MONGO_CONN)

In [36]:
db = mongo_client[db_name]
collection = db["MSFTFY23Q2"]
cursor = collection.find({})
for doc in cursor:
    doc = doc
    print(doc)

{'_id': ObjectId('65b88dbff384be8800d94b03'), 'id': 0, 'content': "Microsoft FY23 Second Quarter Earnings Conference Call\xa0\n\nBrett Iversen, Satya Nadella, Amy Hood\xa0\n\nTuesday, January 24, 2023\n\nBRETT IVERSEN: \n\nGood afternoon and thank you for joining us today. On the call with me are Satya Nadella, chairman and chief executive officer, Amy Hood, chief financial officer, Alice Jolla, chief accounting officer, and Keith Dolliver, deputy general counsel.\n\nOn the Microsoft Investor Relations website, you can find our earnings press release and financial summary slide deck, which is intended to supplement our prepared remarks during today’s call and provides the reconciliation of differences between GAAP and non-GAAP financial measures. \n\nOn this call we will discuss certain non-GAAP items. The non-GAAP financial measures provided should not be considered as a\xa0substitute for or superior to the measures of financial performance prepared in accordance with GAAP.\xa0They ar

In [37]:
doc

{'_id': ObjectId('65baabd13baff198c8a7bbf5'),
 'id': 14,
 'content': 'Sometimes, you will have ISVs who are different, right? If an ISV is optimizing, they optimize and say, what is the new set of features to add at what ramp. All that said, as Amy noted in her remarks, last quarter we added more long-term customer commitments to Azure, and it’s great to see that continue.*\n\nBRETT IVERSEN: That wraps up the Q&A portion of today’s earnings call. Thank you for joining us today, and we look forward to speaking with all of you soon. \n\n(Operator Direction.)\n\nEND OF AUDIO\n\nComplete answer included in transcript but not captured in the webcast',
 'contentVector': [-0.004241057,
  -0.0273975,
  0.018400498,
  -0.016232545,
  -0.015460212,
  0.025907034,
  0.0009942095,
  -0.0160564,
  -0.023711981,
  -0.01646289,
  0.039294142,
  0.037803672,
  0.004213958,
  -0.01695068,
  -0.019470925,
  0.004366392,
  0.014443984,
  0.0123166805,
  0.029538354,
  -0.04241057,
  -0.017208124,
  0.006

In [38]:
doc["source"]

'..\\..\\..\\preprocessing\\DATA\\MSFTTranscriptFY23Q2.docx'

In [39]:
doc["fiscal_quarter"]

'2'